In [1]:
#city list from here: https://en.wikipedia.org/wiki/List_of_cities_proper_by_population

In [2]:
#ningbo city and yangon (rangoon) return no results for some reason

In [3]:
city_list = [
"shanghai",
"karachi",
"beijing",
"lagos",
"delhi",
"tianjin",
"istanbul",
"tokyo",
"guangzhou",
"mumbai",
"moscow",
"sao paulo",
"shenzhen",
"jakarta",
"seoul",
"kinshasa",
"cairo",
"lahore",
"mexico city",
"lima",
"new york city",
"london",
"bengaluru",
"bangkok",
"ho chi minh city",
"dongguan",
"chongqing",
"nanjing",
"tehran",
"shenyang",
"bogota",
"hong kong",
"hanoi",
"baghdad",
"changsha",
"dhaka",
"wuhan",
"hyderabad",
"rio de janeiro",
"faisalabad",
"foshan",
"zunyi",
"santiago",
"riyadh",
"ahmedabad",
"singapore",
"shantou",
"saint petersburg",
"chennai",
"abidjan",
"chengdu",
"alexandria",
"kolkata",
"ankara",
"xi'an",
"surat",
"johannesburg",
"dar es salaam",
"suzhou",
"harbin",
"giza",
"zhengzhou",
"new taipei city",
"los angeles",
"cape town",
"yokohama",
"busan",
"hangzhou",
"xiamen",
"quanzhou",
"berlin",
"rawalpindi",
"jeddah",
"durban",
"hyderabad",
"kabul",
"casablanca",
"hefei",
"pyongyang",
"madrid",
"peshawar",
"ekurhuleni",
"nairobi",
"zhongshan",
"pune",
"addis ababa",
"jaipur",
"buenos aires",
"wenzhou"
]

In [4]:
import requests
rstring = "http://nominatim.openstreetmap.org/search?format=json&polygon=1"

polygon_list = []

for city in city_list:
    r = requests.get(rstring+"&city="+city)
    try:
        polygon_points = r.json()[0]['polygonpoints']
        polygon_list.append(polygon_points)
    except:
        try:
            polygon_points = r.json()[1]['polygonpoints']
            polygon_list.append(polygon_points)
        except:
            try:
                polygon_points = r.json()[2]['polygonpoints']
                polygon_list.append(polygon_points)
            except:
                print city,"err"

In [6]:
len(polygon_list)

89

In [7]:
polygon_list = [[[float(item) for item in pair] for pair in member] for member in polygon_list]

In [8]:
import pandas as pd

In [9]:
df = pd.DataFrame(city_list,columns=['city'])

In [10]:
df['polygon'] = polygon_list

In [11]:
from shapely.geometry import Polygon
df['Polygon'] = df.polygon.map(Polygon)

In [12]:
from numpy import repeat

In [13]:
city_list = []
lons = []
lats = []

for i in range(len(df)):
    city = df.city.loc[i]
    polygon = df.polygon.loc[i]
    n = len(polygon)
    city_list.extend(list(repeat(city,n))) #extend is perfect here
    tmp_lons = [item[0] for item in polygon]
    tmp_lats = [item[1] for item in polygon]
    lons.extend(tmp_lons)
    lats.extend(tmp_lats)

In [14]:
d = pd.DataFrame(city_list,columns=['city'])

In [15]:
d['lons'] = lons
d['lats'] = lats

In [16]:
d.to_csv("./city_points.csv",index=False)

In [17]:
tweets = pd.read_csv("/Users/damoncrockett/Desktop/Twitter_Global_Viz/twitter_metadata_2014_lean.csv")

In [18]:
from shapely.geometry import Point

In [19]:
from rtree import index
idx = index.Index()
count = -1
for item in df.Polygon:
    count +=1
    idx.insert(count, item.bounds)

In [20]:
m = len(tweets)
cities = []
for i in range(m):
    if i in [100000,2000000,400000,800000,1000000,10000000,50000000,100000000]:
        print i
    tmp = 'outside'
    pt = (tweets.lon[i],tweets.lat[i])
    for j in idx.intersection(pt):
        if Point(pt).within(df.Polygon.loc[j]):
            tmp = df.city[j]
            break
    cities.append(tmp)

100000
400000
800000
1000000
2000000
10000000
50000000
100000000


In [21]:
tweets['city'] = cities

In [22]:
import numpy as np

In [24]:
np.mean(tweets.city!='outside')

0.090781326366205534

In [25]:
tweets.city.value_counts()

outside             121685253
jakarta               1410341
istanbul              1342407
tokyo                  954327
los angeles            931673
mexico city            796238
riyadh                 791820
bangkok                791652
sao paulo              650923
madrid                 591974
bogota                 567561
jeddah                 552771
buenos aires           449255
singapore              336808
ankara                 289647
saint petersburg       172519
cairo                  146463
johannesburg           115119
baghdad                 89589
hong kong               89055
berlin                  88398
santiago                79604
delhi                   70021
cape town               68378
lagos                   60353
giza                    56337
nairobi                 53158
alexandria              52264
new taipei city         51916
mumbai                  42837
                      ...    
moscow                   3078
chengdu                  3034
wuhan     

In [27]:
df['area'] = df.Polygon.map(lambda x:x.area)

In [ ]:
'''
Ok this area measure shows the same errors I found with the original openstreetmap data.
Moscow and NYC have these weird tiny polygons, and they are not the only cities like
that either. I'm going to have to just locate the city centers and draw a bounding box
around that. On the face of it, it's less accurate and misses a lot, but honestly these
'polygons' here were just circles anyway.
'''